# Pràctica 1 - El procés de l'aprenentatge automàtic:
## Imports

In [1]:
import os
from sklearn.calibration import LabelEncoder
from sklearn.discriminant_analysis import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from skimage import io
from skimage.transform import resize
from skimage.color import rgb2gray
import numpy as np

## Funció de processament de dades

In [2]:
def load_and_preprocess(folder_path, target_size=(64, 64)):
    images = []
    labels = []
    
    for label in os.listdir(folder_path):
        label_path = os.path.join(folder_path, label)
        
        for image_name in os.listdir(label_path):
            image_path = os.path.join(label_path, image_name)
            image = io.imread(image_path)
            
            # Preprocess the image (resize, convert to grayscale, etc.)
            image = resize(image, target_size)
            # Check if the image is already grayscale
            if image.ndim == 2:
                # Grayscale image
                images.append(image.flatten())
            else:
                # Convert color image to grayscale
                gray_image = rgb2gray(image)
                images.append(gray_image.flatten())

            labels.append(label)
    
    return np.array(images), np.array(labels)

## Processament de dades

In [3]:
train_path = 'dat/train/'
test_path = 'dat/test/'
X_train, y_train = load_and_preprocess(train_path)
print("Imatges train processades")
X_test, y_test = load_and_preprocess(test_path)
print("Imatges test processades")

Imatges train processades
Imatges test processades


## Transformació de les dades

In [4]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
#label_encoder = LabelEncoder()
#y_train = label_encoder.fit_transform(y_train)
#y_test = label_encoder.fit_transform(y_test)
print("Imatges escalades")

Imatges escalades


## Entrenament i test dels SVM

### Kernel Lineal

In [6]:
print("KERNEL LINEAL")
svm = SVC(kernel='linear')
svm.fit(X_train_scaled, y_train)
print("Model entrenat")
y_pred = svm.predict(X_test_scaled)

# Evaluate the Model
cf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cf_matrix)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

KERNEL LINEAL
Model entrenat
Confusion matrix:
 [[24  3 12  3  3  4  6 14  5  9  5  2  8 11  7]
 [ 7 48  2  2  1  2 14 14  2  3  6  5  0  3  6]
 [ 8  3 81  4 13  6 12 89 21 30  6 13  7  4 13]
 [20  5 23 12 14  8 18 25 25  9  6 13 14  8 15]
 [ 7  2 23 10 74 15  6 17  8  6  8  3  4  7  2]
 [ 8  2 16  3 18 41  5  5  8  6  2  4  3 11  9]
 [16 20 11  5  8  2 61 36 23  8 19 11  6 12 18]
 [10  4 45  4  4  8 15 91 17 39  5  0  1  1 16]
 [ 5  6 26 10 12  1 27 30 51  9  6 14  4  8 19]
 [ 4  0 35  1  6  4  1 33  5 63  2  1  2  0  3]
 [16 16 16  4 12  3 17 30 14 21 26  9  4 12 11]
 [ 8  7 38  6  7  5 12 18 35 10 15 24  9  6  8]
 [14  8 11  4  3  7  4 12 10  7  5  6  9  6  4]
 [22  7 11  5  6 13  8 17 12 16  7  8 12 39  6]
 [25  7 33  5 11  9 22 55 29 33 13  7  4  1 20]]
Accuracy:  0.22244556113902847


### Kernel RBF

In [5]:
print("KERNEL RBF")
svm = SVC(kernel='rbf')
svm.fit(X_train_scaled, y_train)
print("Model entrenat")
y_pred = svm.predict(X_test_scaled)

# Evaluate the Model
cf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cf_matrix)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

KERNEL RBF
Model entrenat
Confusion matrix:
 [[ 34   3   0   5   2   3   5   8   5  10  13   3  16   6   3]
 [ 15  43   0   3   0   0  16   2   8   2  11   2   6   6   1]
 [  6   1  96   5   3   9  10  26  61  57   3  15   7   3   8]
 [ 20   5   4  48   6   9  26   2  28   5   1  15  31  13   2]
 [  6   3  11  11  94  13   2   4  12   7   9   7   8   3   2]
 [  6   0   2   6   3  95   3   1   4   1   5   0   7   7   1]
 [ 17  20   7  15   1   2  62   5  45  11  22   9  27   6   7]
 [  9   5  36   7   1   5  21  45  37  61   4   6   5   1  17]
 [  5  11  15   8   5   4  29   3 105   6   7  15   6   4   5]
 [  5   3  13   1   1   5   0   3  10 112   2   1   1   0   3]
 [ 24   6  16   9   5  13  12   2  18  16  58   6  14   8   4]
 [  5   5  22  14   2  13  13   4  52   7   7  37  18   7   2]
 [ 15   9   2  13   1   4   5   2  10   5  10   6  24   4   0]
 [ 24   8   2  18   5  11  15   6   8  11  18   3  28  29   3]
 [ 16  11  21  19   5   6  22  18  34  24  15  14  17   6  46]]
Accuracy:

### Kernel Polinomic

In [8]:
print("KERNEL POLINOMIC")
svm = SVC(kernel='poly')
svm.fit(X_train_scaled, y_train)
print("Model entrenat")
y_pred = svm.predict(X_test_scaled)

# Evaluate the Model
cf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cf_matrix)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

KERNEL POLINOMIC
Confusion matrix:
 [[ 11   2  54   4   1   0   1   0   1   2  16   2  15   2   5   0]
 [ 35  14 128  20   6   2  21   0   1   6   4   3   7   1  12   0]
 [  5  56  57  42   2   4  12   1   0  21   2   0  13   0  13   0]
 [  4   3 131   5   3   0   7   1   0   1   2   0   2   0   1   0]
 [ 14  12  76  21   0   2  25   0   2  10   6  24  11   3   5   0]
 [  8  18  79  55   2   2  18   2   3   2   0   2  10   0   7   0]
 [  7   1  48  17   0   3   2   0   0   2   2   1  23   1   3   0]
 [ 11   7  78  16   1   4   4   2   0   6   8   0  25  21   6   0]
 [  5   7  64  14   1   2   3   2   0   6   7   0  18  20   5   0]
 [ 25  21  90  29  16   2  21   5   2  19   3   4  16   2  19   0]
 [  5  10  34  13   0  28   0   0   0   8   3   2  10   1   1   0]
 [ 18  12 131  46   4   0  67   1   2   6   0   2   7   0  14   0]
 [  8  13  90  40   2   0   9   0   1  10   3   0  23   0  16   0]
 [  5   5  70  14   0   3  21  58   3   1   2   1   8   0   1   0]
 [  3   8  46  12   0   0 